# Detecting bad posture

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import Adam
import numpy as np

2022-01-03 23:53:30.232690: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/mauriciomani/Desktop/rowing_tracker/venv/lib/python3.8/site-packages/cv2/../../lib64:
2022-01-03 23:53:30.232708: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
BATCH_SIZE = 16 # 10, 8, 16
ROTATION = 0.05

In [3]:
idg = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input, validation_split=0.1)
train = idg.flow_from_directory(directory="../img/train",
                                shuffle=True,
                                target_size=(224, 224), 
                                classes=["good", "bad"], 
                                batch_size=BATCH_SIZE,
                                subset='training')

val = idg.flow_from_directory(directory="../img/train",
                                target_size=(224, 224), 
                                classes=["good", "bad"], 
                                batch_size=BATCH_SIZE,
                                subset='validation')

Found 1045 images belonging to 2 classes.
Found 115 images belonging to 2 classes.


In [4]:
#vgg16_model = tf.keras.applications.vgg16.VGG16()
resnet_model = tf.keras.applications.resnet50.ResNet50()

2022-01-03 22:39:24.126406: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-03 22:39:24.132884: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-01-03 22:39:24.177654: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-03 22:39:24.177848: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1650 Ti with Max-Q Design computeCapability: 7.5
coreClock: 1.2GHz coreCount: 16 deviceMemorySize: 3.82GiB deviceMemoryBandwidth: 149.04GiB/s
2022-01-03 22:39:24.177907: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file

In [5]:
def count_params(model):
    non_trainable_params = np.sum([np.prod(v.get_shape().as_list()) for v in model.non_trainable_weights])
    trainable_params = np.sum([np.prod(v.get_shape().as_list()) for v in model.trainable_weights])
    return {"non_trainable_params": non_trainable_params, "trainable_params": trainable_params}

In [6]:
params = count_params(resnet_model)
print(params)

{'non_trainable_params': 53120, 'trainable_params': 25583592}


In [21]:
#model = Sequential()
#for layer in resnet_model.layers[:-1]:
#    model.add(layer)

In [7]:
for layer in resnet_model.layers[:143]:
    layer.trainable = False

In [ ]:
#for layer in model.layers:
#    layer.trainable = False

In [8]:
model = Sequential()
model.add(resnet_model)
model.add(Flatten())
model.add(Dense(units=2, activation="softmax"))

In [8]:
#model.add(Dense(units=2, activation="softmax"))

In [9]:
print(count_params(model))

{'non_trainable_params': 8611712, 'trainable_params': 17027002}


In [10]:
# Prove on decreasing learning rate
model.compile(optimizer=Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [11]:
model.fit(x=train, validation_data=val, epochs=5, shuffle=True, class_weight={0:0.2, 1:0.8})

2022-01-03 22:39:44.760686: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-01-03 22:39:44.779675: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2400000000 Hz


Epoch 1/5
66/66 [==============================] - 92s 1s/step - loss: 0.2025 - precision: 0.9370 - recall: 0.9370 - val_loss: 0.6234 - val_precision: 0.9826 - val_recall: 0.9826
Epoch 2/5
66/66 [==============================] - 90s 1s/step - loss: 0.1900 - precision: 0.9804 - recall: 0.9804 - val_loss: 0.6159 - val_precision: 0.9652 - val_recall: 0.9652
Epoch 3/5
66/66 [==============================] - 90s 1s/step - loss: 0.1913 - precision: 0.9788 - recall: 0.9788 - val_loss: 0.6045 - val_precision: 0.9913 - val_recall: 0.9913
Epoch 4/5
66/66 [==============================] - 90s 1s/step - loss: 0.1923 - precision: 0.9782 - recall: 0.9782 - val_loss: 0.5958 - val_precision: 0.9913 - val_recall: 0.9913
Epoch 5/5
66/66 [==============================] - 92s 1s/step - loss: 0.1779 - precision: 0.9849 - recall: 0.9849 - val_loss: 0.5875 - val_precision: 0.9913 - val_recall: 0.9913


In [12]:
model.save("../models/resnet.h5")

In [4]:
from tensorflow.keras.models import load_model

model = load_model("../models/resnet.h5")

2022-01-03 23:55:34.203462: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-03 23:55:34.209056: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-01-03 23:55:34.243532: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-03 23:55:34.243755: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1650 Ti with Max-Q Design computeCapability: 7.5
coreClock: 1.2GHz coreCount: 16 deviceMemorySize: 3.82GiB deviceMemoryBandwidth: 149.04GiB/s
2022-01-03 23:55:34.243823: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file